## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-02-28-35 +0000,bbc,China growth slows as trade turmoil weighs on ...,https://www.bbc.com/news/articles/c20r461g61xo
1,2025-07-15-02-15-17 +0000,nypost,Three Democrats win double-digit support in ea...,https://nypost.com/2025/07/14/us-news/three-de...
2,2025-07-15-02-08-51 +0000,bbc,Freeholders begin High Court challenge over re...,https://www.bbc.com/news/articles/c3vd23g29deo
3,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o
4,2025-07-15-01-29-35 +0000,nypost,Former Obama speechwriter admits shunning cons...,https://nypost.com/2025/07/14/us-news/former-o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,11
28,russia,5
81,india,4
53,claims,4
229,he,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...,32
29,2025-07-14-19-33-55 +0000,nypost,First lady Melania Trump helped convince presi...,https://nypost.com/2025/07/14/us-news/first-la...,31
42,2025-07-14-15-54-37 +0000,nypost,Trump claims ‘not sure we need’ bipartisan san...,https://nypost.com/2025/07/14/us-news/trump-cl...,27
3,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,25
28,2025-07-14-19-51-43 +0000,nypost,Supreme Court lets Trump move forward with fir...,https://nypost.com/2025/07/14/us-news/supreme-...,23


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,32,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...
60,20,2025-07-14-05-26-09 +0000,nypost,Biden admits he didn’t approve each individual...,https://nypost.com/2025/07/14/us-news/biden-de...
5,19,2025-07-15-01-25-19 +0000,nypost,Democratic city officers flocking to cop-frien...,https://nypost.com/2025/07/14/us-news/blue-cit...
30,17,2025-07-14-19-28-42 +0000,nypost,Andrew Cuomo confirms independent run for NYC ...,https://nypost.com/2025/07/14/us-news/andrew-c...
46,17,2025-07-14-14-55-07 +0000,bbc,"Boeing fuel switches safe, regulator says afte...",https://www.bbc.com/news/articles/ce9xpgnx3vdo
47,15,2025-07-14-14-31-32 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o
24,14,2025-07-14-20-37-28 +0000,nyt,The Economy Has Been Resilient. The New Round ...,https://www.nytimes.com/2025/07/14/us/politics...
63,13,2025-07-14-03-50-12 +0000,nypost,JD Vance claps back at snide comment from Gov....,https://nypost.com/2025/07/13/us-news/jd-vance...
22,12,2025-07-14-20-57-24 +0000,nypost,Sen. Rand Paul renews push for criminal probe ...,https://nypost.com/2025/07/14/us-news/sen-rand...
28,12,2025-07-14-19-51-43 +0000,nypost,Supreme Court lets Trump move forward with fir...,https://nypost.com/2025/07/14/us-news/supreme-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
